# To do
* Detrend Forcing data 
    * Remove overall trend
    * Remove seasonal trend


## Load packages

In [7]:
from matplotlib import pyplot as plt
import numpy as np
import netCDF4 as nc
import pandas as pd
import xarray as xr

## Load in current forcing data

In [8]:
years = np.arange(2000, 2024, 1)
P_data = []
R_data = []
T_data = []
lai_data = []
calibration_time = [2000, 2010]

for year in years:
    file_path = 'data/total_precipitation/tp.daily.calc.era5.0d50_CentralEurope.' + str(year) + '.nc'
    P_data.append(xr.open_dataset(file_path)['tp'])

    file_path = 'data/net_radiation/nr.daily.calc.era5.0d50_CentralEurope.' + str(year) + '.nc'
    R_data.append(xr.open_dataset(file_path)['nr'])

    file_path = 'data/daily_average_temperature/t2m_mean.daily.calc.era5.0d50_CentralEurope.' + str(year) + '.nc'
    T_data.append(xr.open_dataset(file_path)['t2m'])

    file_path = 'data/lai/lai.daily.0d50_CentralEurope.' + str(year) + '.nc'
    lai_data.append(xr.open_dataset(file_path)['lai'])

# Convert lists to xarray datasets
P_data = xr.concat(P_data, dim='time')
R_data = xr.concat(R_data, dim='time')
T_data = xr.concat(T_data, dim='time')
lai_data = xr.concat(lai_data, dim='time')

In [9]:
R_data

<xarray.DataArray 'nr' (time: 8766, lat: 22, lon: 22)> Size: 34MB
array([[[-4204685.4375   , -4146013.3125   , -4072473.5625   , ...,
         -1185969.09375  , -1515929.25     , -1654409.0625   ],
        [-4076561.25     , -3962469.375    , -3814689.375    , ...,
         -1374381.1875   , -1532713.125    , -1769577.1875   ],
        [-3831017.4375   , -3633133.5      , -3416757.5625   , ...,
         -1031869.3125   , -1137005.34375  , -1223857.21875  ],
        ...,
        [  840718.59375  ,   639290.71875  ,   220346.8125   , ...,
          -246661.5      ,  -189137.625    ,  -403797.75     ],
        [  816938.625    ,   392638.6875   ,    35778.75     , ...,
         -2317972.875    , -1409253.       ,  -729534.       ],
        [  279334.6875   ,   -87929.4375   ,  -213557.0625   , ...,
         -4740881.25     , -3105153.       , -1112485.875    ]],

       [[-4145180.4375   , -4166248.21875  , -4212140.25     , ...,
         -1143540.375    ,  -938248.3125   ,  -665988.28125  ],
        [-3448088.25     , -3450624.375    , -3428368.21875  , ...,
         -1086396.5625   ,  -676276.3125   ,  -485924.15625  ],
        [-2787824.625    , -2776816.5      , -2676800.4375   , ...,
          -215508.28125  ,    41763.75     ,    79679.8125   ],
...
        [ -761835.375    ,  -587406.75     ,  -311798.625    , ...,
           807873.375    ,   345277.5      ,  -431358.375    ],
        [ -489467.25     ,  -402094.875    ,  -392335.125    , ...,
           241369.5      ,   583909.5      ,   289273.5      ],
        [  141377.25     ,   322265.25     ,    67788.75     , ...,
         -1337366.625    ,  -460618.6875   ,   425005.3125   ]],

       [[-2403931.734375 , -2448631.828125 , -2513911.875    , ...,
         -2067191.4375   , -2509163.625    , -2706635.71875  ],
        [-2575939.78125  , -2683055.625    , -2764663.78125  , ...,
         -1728491.625    , -2132759.53125  , -2655215.625    ],
        [-2473223.71875  , -2727447.75     , -2914987.78125  , ...,
         -1168923.75     , -1394031.65625  , -1849155.46875  ],
        ...,
        [-2707195.875    , -2462479.6875   , -1806571.6875   , ...,
          1333292.203125 ,  1376188.125    ,   869240.25     ],
        [-1698439.6875   , -1687247.625    , -1164959.4375   , ...,
           583408.5      ,  1254396.375    ,  1244512.125    ],
        [ -764823.75     ,  -816231.75     ,  -708935.53125  , ...,
          -913151.625    ,   119752.3125   ,  1127976.375    ]]])
Coordinates:
  * lon      (lon) float64 176B 4.75 5.25 5.75 6.25 ... 13.75 14.25 14.75 15.25
  * lat      (lat) float64 176B 55.25 54.75 54.25 53.75 ... 45.75 45.25 44.75
  * time     (time) datetime64[ns] 70kB 2000-01-01 2000-01-02 ... 2023-12-31
Attributes:
    long_name:  Net radiation
    units:      J m-2

In [3]:
R_data = np.concatenate(R_data)
P_data = np.concatenate(P_data)
T_data = np.concatenate(T_data)
lai_data = np.concatenate(lai_data)

In [4]:
R_data

masked_array(
  data=[[[-4204685.4375   , -4146013.3125   , -4072473.5625   , ...,
          -1185969.09375  , -1515929.25     , -1654409.0625   ],
         [-4076561.25     , -3962469.375    , -3814689.375    , ...,
          -1374381.1875   , -1532713.125    , -1769577.1875   ],
         [-3831017.4375   , -3633133.5      , -3416757.5625   , ...,
          -1031869.3125   , -1137005.34375  , -1223857.21875  ],
         ...,
         [  840718.59375  ,   639290.71875  ,   220346.8125   , ...,
           -246661.5      ,  -189137.625    ,  -403797.75     ],
         [  816938.625    ,   392638.6875   ,    35778.75     , ...,
          -2317972.875    , -1409253.       ,  -729534.       ],
         [  279334.6875   ,   -87929.4375   ,  -213557.0625   , ...,
          -4740881.25     , -3105153.       , -1112485.875    ]],

        [[-4145180.4375   , -4166248.21875  , -4212140.25     , ...,
          -1143540.375    ,  -938248.3125   ,  -665988.28125  ],
         [-3448088.25     , -345